# **PERCEPTRON**

this script is for setup, execution, and evaluation of the perceptron algorithm

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
'''
Import data from data_organization.ipynb
'''
import pandas as pd
import numpy as np
import funcs as f

df_all = pd.read_csv('../data/df_all.csv').drop('Unnamed: 0', axis = 1)
df_gus = pd.read_csv('../data/df_gus.csv').drop('Unnamed: 0', axis = 1)
df_tgus = pd.read_csv('../data/df_tgus.csv').drop('Unnamed: 0', axis = 1)
df_tgus_st = pd.read_csv('../data/df_tgus*.csv').drop('Unnamed: 0', axis = 1)
df_raw = pd.read_csv('../data/df_raw.csv').drop('Unnamed: 0', axis = 1)

In [6]:
'''
run perceptron for 100 different random combinations of train/test
save the best weight vector and use it later...
'''
#run the test for 30 different random numbers at np.random.seed()
avg_accuracy = []
for i in range(100):
    np.random.seed(8)
    df_final=df_final.sample(frac=1) 
    train_proportion = 0.8 
    n = len(df_final)
    t = int(train_proportion * n)

    #separate features and target
    detected = df_final['detected']
    features = df_final.loc[:, ~df_final.columns.isin(['detected'])]

    #features in training set
    train_x = features.iloc[:t,:].reset_index().iloc[:,1:]
    #features in test set
    test_x = features.iloc[t:,:].reset_index().iloc[:,1:]
    #targets in train set
    train_y = detected[:t].reset_index().iloc[:,1:]
    #targets in test set
    test_y = detected[t:].reset_index().iloc[:,1:]


    '''
    run perceptron!
    '''
    w = f.perceptron(X=train_x, y = train_y)# maxchecks= 50000, maxiters = 50000)#, maxchecks = 500000, maxiters = 500000)


    '''
    calculate losses
    '''
    losses = 0
    test_losses = 0
    train_losses = 0
    for idx, row in test_x.iterrows():
        result = f.sign(np.dot(w, row))

        if result != test_y.iloc[idx,0]:
            losses += 1

    avg_accuracy = np.append(round((1 - losses / len(test_x)) * 100, 2), avg_accuracy)
    
    if avg_accuracy[0] >= np.max(avg_accuracy):
        print(avg_accuracy[0])
        w_best = w

print('Average Accuracy: ' + str(round(np.mean(avg_accuracy),2)) + '%' )
print('Highest Accuracy: ' + str(round(np.max(avg_accuracy),2)) + '%' )
print('Lowest Accuracy: ' + str(round(np.min(avg_accuracy),2)) + '%' )
    

90.24
90.24
93.9
93.9
96.34
96.34
96.34
96.34
Average Accuracy: 87.75%
Highest Accuracy: 96.34%
Lowest Accuracy: 68.29%


In [7]:
''' 
now test w_best on different test sets of half the data size
'''
post_avg_accuracy = []
for i in range(100):
    np.random.seed(8)
    df_final=df_final.sample(frac=1) 
    train_proportion = .5
    n = len(df_final)
    t = int(train_proportion * n)

    #separate features and target
    detected = df_final['detected']
    features = df_final.loc[:, ~df_final.columns.isin(['detected'])]

    #features in test set
    post_test_x = features.iloc[t:,:].reset_index().iloc[:,1:]
    #targets in test set
    post_test_y = detected[t:].reset_index().iloc[:,1:]

    '''
    calculate losses
    '''
    #0-1 loss
    losses = 0
    for idx, row in post_test_x.iterrows():
        result = f.sign(np.dot(w_best, row))

        if result != post_test_y.iloc[idx,0]:
            losses += 1
            
    post_avg_accuracy = np.append(round((1 - losses / len(post_test_x)) * 100, 2), post_avg_accuracy)
    
print('Average Accuracy: ' + str(round(np.mean(post_avg_accuracy),2)) + '%' )
print('Highest Accuracy: ' + str(round(np.max(post_avg_accuracy),2)) + '%' )
print('Lowest Accuracy: ' + str(round(np.min(post_avg_accuracy),2)) + '%' )


Average Accuracy: 87.83%
Highest Accuracy: 91.13%
Lowest Accuracy: 84.73%
